In [ ]:
!git clone git@github.com:chirag-wadhwa/Pop.git

In [ ]:
import os
os.chdir('musicautobot')

In [ ]:
!apt install musescore fluidsynth
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2

In [ ]:
!pip install torch fastai music21 pebble fluidsynth midi2audio

In [ ]:
from musicautobot.numpy_encode import *
from musicautobot.utils.file_processing import process_all, process_file
from musicautobot.config import *
from musicautobot.music_transformer import *
from musicautobot.utils.setup_musescore import setup_musescore
setup_musescore()

In [ ]:
from midi2audio import FluidSynth
from IPython.display import Audio

In [ ]:
# Colab cannot play music directly from music21 - must convert to .wav first
def play_wav(stream):
    out_midi = stream.write('midi')
    out_wav = str(Path(out_midi).with_suffix('.wav'))
    FluidSynth("font.sf2").midi_to_audio(out_midi, out_wav)
    return Audio(out_wav)


In [ ]:
# Location of your midi files
midi_path =  Path('data/midi/examples')

# Location of saved datset
data_path = Path('data/numpy')

In [ ]:
# Data
data = MusicDataBunch.empty(data_path)
vocab = data.vocab

# For Saved Data:
# data = load_data(data_path, 'musicitem_data_save.pkl')

In [ ]:
# Pretrained Model
pretrained_url = 'https://ashaw-midi-web-server.s3-us-west-2.amazonaws.com/pretrained/MusicTransformerKeyC.pth'
# pretrained_url = 'https://ashaw-midi-web-server.s3-us-west-2.amazonaws.com/pretrained/MusicTransformer.pth'

pretrained_path = data_path/'pretrained'/Path(pretrained_url).name
pretrained_path.parent.mkdir(parents=True, exist_ok=True)
download_url(pretrained_url, dest=pretrained_path)

In [ ]:
# Learner
learn = music_model_learner(data, pretrained_path=pretrained_path)

In [ ]:
# Learner
learn = music_model_learner(data, pretrained_path=pretrained_path)

In [ ]:
idx = 1
f = midi_files[idx]; f

In [ ]:
cutoff_beat = 10

item = MusicItem.from_file(f, data.vocab)
seed_item = item.trim_to_beat(cutoff_beat)

In [ ]:
seed_item.show()

In [ ]:
# seed_item.play()
play_wav(seed_item.stream)

In [ ]:
pred, full = learn.predict(seed_item, n_words=400, temperatures=(1.1,0.4), min_bars=12, top_k=24, top_p=0.7)

In [ ]:
pred.show()

In [ ]:
play_wav(pred.stream)

In [ ]:
full_song = seed_item.append(pred); full_song.show()

In [ ]:
play_wav(full_song.stream)